<a href="https://colab.research.google.com/github/Amisha1019/Image-Colorization-and-Generative-AI/blob/main/Task_2_Colorization_Internship.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import argparse
from pathlib import Path
from glob import glob
import random
import numpy as np
from tqdm import tqdm
from PIL import Image


import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from skimage import color, exposure

from datasets import load_dataset
import numpy as np
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T


In [2]:
IMG_SIZE = 256
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NBINS_DEFAULT = 16

In [3]:
try:
    from datasets import load_dataset
except Exception:
    load_dataset = None

In [12]:

datasets = api.list_datasets(search="/content/HistoricalColor-ECCV2012-DecadeDatabase.tar")

if datasets:
    print("Found datasets:")
    for dataset in datasets:
        print(f"- {dataset.id}")
else:
    print("No datasets found matching the search term.")

Found datasets:


In [13]:
class HistoricalColorizationDataset(Dataset):
    def __init__(self, hf_dataset, image_size=256):
        self.data = hf_dataset
        self.transform_color = T.Compose([
            T.Resize((image_size, image_size)),
            T.ToTensor()               # returns (C, H, W)
        ])
        self.transform_gray = T.Compose([
            T.Resize((image_size, image_size)),
            T.Grayscale(num_output_channels=1),
            T.ToTensor()
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        record = self.data[idx]

        img = record["image"]
        period = record["label"]  # decade (0–?) numeric label

        # Convert to PIL if needed
        if not isinstance(img, Image.Image):
            img = Image.fromarray(np.uint8(img))

        # Create color image tensor
        color_img = self.transform_color(img)

        # Create grayscale image tensor
        gray_img = self.transform_gray(img)

        return gray_img, color_img, period  # Input, Target, Decade Label



In [14]:
datasets = api.list_datasets(search="/content/HistoricalColor-ECCV2012-DecadeDatabase.tar")
print(type(datasets))


<class 'generator'>


In [15]:
try:
    files = api.list_repo_files("/content/HistoricalColor-ECCV2012-DecadeDatabase.tar", repo_type="dataset")

    print("Files in the repository:")
    for file in files:
        print(f"- {file}")
except Exception as e:
    print(f"An error occurred: {e}")

An error occurred: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/content/HistoricalColor-ECCV2012-DecadeDatabase.tar'. Use `repo_type` argument if needed.


In [16]:
class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch, bn=True):
        super().__init__()
        layers = [nn.Conv2d(in_ch, out_ch, 3, padding=1), nn.ReLU(inplace=True)]
        if bn: layers.append(nn.BatchNorm2d(out_ch))
        layers += [nn.Conv2d(out_ch, out_ch, 3, padding=1), nn.ReLU(inplace=True)]
        if bn: layers.append(nn.BatchNorm2d(out_ch))
        self.net = nn.Sequential(*layers)

    def forward(self,x):
        return self.net(x)

In [17]:
class UNetConditional(nn.Module):
    def __init__(self, num_periods=6, emb_dim=128):
        super().__init__()
        self.enc1 = ConvBlock(1,64); self.pool = nn.MaxPool2d(2)
        self.enc2 = ConvBlock(64,128)
        self.enc3 = ConvBlock(128,256)
        self.enc4 = ConvBlock(256,512)
        self.bottleneck = ConvBlock(512,512)
        self.period_emb = nn.Embedding(num_periods, emb_dim)
        self.embed_fc = nn.Sequential(nn.Linear(emb_dim, 512), nn.ReLU())
        self.up4 = nn.ConvTranspose2d(1024,512,2,stride=2)
        self.dec4 = ConvBlock(1024,256)
        self.up3 = nn.ConvTranspose2d(512,256,2,stride=2)
        self.dec3 = ConvBlock(512,128)
        self.up2 = nn.ConvTranspose2d(256,128,2,stride=2)
        self.dec2 = ConvBlock(256,64)
        self.final = nn.Sequential(nn.Conv2d(128,64,3,padding=1), nn.ReLU(), nn.Conv2d(64,2,1))

In [18]:
def forward(self, L, pid):
    # L: (B,1,H,W)
    e1 = self.enc1(L)
    e2 = self.enc2(self.pool(e1))
    e3 = self.enc3(self.pool(e2))
    e4 = self.enc4(self.pool(e3))
    b = self.bottleneck(self.pool(e4))
    emb = self.period_emb(pid)
    emb = self.embed_fc(emb).unsqueeze(-1).unsqueeze(-1)
    b = b + emb
    d4 = self.up4(torch.cat([b, e4], dim=1))
    d4 = self.dec4(torch.cat([d4, e3], dim=1))
    d3 = self.up3(d4); d3 = self.dec3(torch.cat([d3, e2], dim=1))
    d2 = self.up2(d3); d2 = self.dec2(torch.cat([d2, e1], dim=1))
    out = self.final(torch.cat([d2, L], dim=1))
    return out

In [19]:
def lab_from_tensor_rgb(img_tensor):
    # img_tensor: (B,3,H,W) in [0,1]
    arr = img_tensor.permute(0,2,3,1).cpu().numpy()
    labs = []
    for i in range(arr.shape[0]):
        lab = color.rgb2lab(arr[i]).astype(np.float32)
        labs.append(lab)
    labs = np.stack(labs)
    # return L [0,1], ab scaled to [-1,1]
    L = (labs[...,0:1] / 100.0)
    ab = (labs[...,1:] / 128.0)
    L_t = torch.from_numpy(L).permute(0,3,1,2).float()
    ab_t = torch.from_numpy(ab).permute(0,3,1,2).float()
    return L_t, ab_t

In [20]:
def tensor_rgb_from_lab(L_t, ab_t):
    # L_t: (B,1,H,W) [0,1], ab_t: (B,2,H,W) in [-1,1]
    L_np = (L_t.detach().cpu().numpy() * 100.0).transpose(0,2,3,1)
    ab_np = (ab_t.detach().cpu().numpy() * 128.0).transpose(0,2,3,1)
    lab = np.concatenate([L_np, ab_np], axis=-1)
    rgbs = []
    for i in range(lab.shape[0]):
        rgb = color.lab2rgb(lab[i])
        rgbs.append(rgb.clip(0,1))
    rgb_t = torch.tensor(np.stack(rgbs)).permute(0,3,1,2).float()
    return rgb_t

In [21]:
def compute_ab_hist_numpy(ab_batch, nbins=16):
    # ab_batch: (B,2,H,W) torch
    arr = ab_batch.permute(0,2,3,1).cpu().numpy()
    B = arr.shape[0]
    bins = np.linspace(-1,1,nbins+1)
    hist_list = []
    for i in range(B):
        Hmat,_,_ = np.histogram2d(arr[i][:, :, 0].ravel(), arr[i][:, :, 1].ravel(), bins=bins)
        Hmat = Hmat / (Hmat.sum() + 1e-9)
        hist_list.append(Hmat.flatten())
    return np.stack(hist_list)

In [22]:
class PerceptualLoss(nn.Module):
    def __init__(self, device=DEVICE):
        super().__init__()
        vgg = models.vgg16(pretrained=True).features[:16].to(device).eval()
        for p in vgg.parameters(): p.requires_grad = False
        self.vgg = vgg
        self.preprocess = transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
    def forward(self, pred, target):
        # pred, target: (B,3,H,W) in [0,1]
        x = self.preprocess(pred.clone())
        y = self.preprocess(target.clone())
        fx = self.vgg(x)
        fy = self.vgg(y)
        return nn.functional.l1_loss(fx, fy)

In [23]:
def train_loop(model, dataloader, optimizer, epoch, period_hist=None, nbins=16):
    model.train()
    perc = PerceptualLoss().to(DEVICE)
    for batch in tqdm(dataloader, desc=f"Epoch {epoch}"):
        gray, color, pid = batch
        gray = gray.to(DEVICE); color = color.to(DEVICE); pid = pid.to(DEVICE).long()
        # convert color to Lab
        L_gt, ab_gt = lab_from_tensor_rgb(color)
        L_gt = L_gt.to(DEVICE); ab_gt = ab_gt.to(DEVICE)
        # model predict
        ab_pred = model(L_gt, pid)
        # losses
        l1 = nn.functional.l1_loss(ab_pred, ab_gt)
        rgb_pred = tensor_rgb_from_lab(L_gt, ab_pred).to(DEVICE)
        rgb_gt = tensor_rgb_from_lab(L_gt, ab_gt).to(DEVICE)
        p_loss = perc(rgb_pred, rgb_gt)
        # histogram loss (simple MSE to period histogram)
        hist_pred = compute_ab_hist_numpy(ab_pred.detach().cpu(), nbins=nbins)
        hist_target = np.array([period_hist[int(p.item())] for p in pid.cpu()]) if period_hist is not None else np.ones_like(hist_pred)
        hist_loss = torch.tensor(((hist_pred - hist_target)**2).mean()).to(DEVICE)
        loss = 1.0*l1 + 0.1*p_loss + 0.5*hist_loss
        optimizer.zero_grad(); loss.backward(); optimizer.step()

In [25]:
def inference_image(model, img_path, period_id=0, img_size=IMG_SIZE, save_path=None):
    img = Image.open('/content/HistoricalColor-ECCV2012-DecadeDatabase.tar').convert('RGB')
    t_color = transforms.Compose([transforms.Resize((img_size,img_size)), transforms.ToTensor()])
    t_gray = transforms.Compose([transforms.Resize((img_size,img_size)), transforms.Grayscale(num_output_channels=1), transforms.ToTensor()])
    color_t = t_color(img).unsqueeze(0)
    gray_t = t_gray(img).unsqueeze(0)
    # convert color to lab to get L
    L, _ = lab_from_tensor_rgb(color_t)
    L = L.to(DEVICE)
    pid = torch.tensor([period_id], device=DEVICE)
    model.eval()
    with torch.no_grad():
        ab_pred = model(L, pid)
    rgb = tensor_rgb_from_lab(L.cpu(), ab_pred.cpu())[0].permute(1,2,0).numpy()
    # upscale back to original size
    out_img = Image.fromarray((rgb*255).astype('uint8'))
    out_img = out_img.resize(img.size)
    if save_path:
        out_img.save(save_path)
    return out_img

In [26]:
def build_period_histograms(hf_split, nbins=NBINS_DEFAULT, out_npz='period_histograms.npz'):
    # Compute aggregated ab histogram per label
    labels = {}
    counts = {}
    for rec in tqdm(hf_split, desc='Computing histograms'):
        img = rec['/content/HistoricalColor-ECCV2012-DecadeDatabase.tar']
        label = int(rec.get('label', 0))
        if not isinstance(img, Image.Image):
            img = Image.fromarray(np.uint8(img))
        img = img.resize((IMG_SIZE, IMG_SIZE))
        arr = np.array(img) / 255.0
        lab = color.rgb2lab(arr.astype(np.float32))
        ab = lab[...,1:]
        # scale to [-1,1]
        ab = ab / 128.0
        Hmat, xedges, yedges = np.histogram2d(ab[:,:,0].ravel(), ab[:,:,1].ravel(), bins=nbins, range=[[-1,1],[-1,1]])
        Hmat = Hmat / (Hmat.sum() + 1e-9)
        if label not in labels:
            labels[label] = Hmat
            counts[label] = 1
        else:
            labels[label] = labels[label] + Hmat
            counts[label] += 1
    # normalize per label
    out = {}
    for k,v in labels.items():
        v = v / (counts[k] + 1e-9)
        out[k] = v.flatten()
    np.savez(out_npz, **out)
    print(f"Saved histograms to {out_npz}")
    return out

In [32]:
from datasets import load_dataset
import os

# Before running this cell, make sure to extract the .tar file
# For example, you can use the 'tarfile' library:
# import tarfile
# with tarfile.open("/content/HistoricalColor-ECCV2012-DecadeDatabase.tar", 'r') as tar:
#     tar.extractall("/content/historical_dataset")

# Then, replace the data_dir with the path to the extracted directory
try:
    dataset = load_dataset(
        "imagefolder",
        data_dir="/content/extracted_dataset_directory" # Replace with the actual extracted directory path
    )

    print("Image dataset loaded successfully!")
    print(dataset)

except Exception as e:
    print(f"Error loading dataset: {e}")
    print("Please ensure the .tar file is extracted and the data_dir points to the correct directory.")

Error loading dataset: The directory at /content/extracted_dataset_directory doesn't contain any data files
Please ensure the .tar file is extracted and the data_dir points to the correct directory.


In [33]:
def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('--mode', choices=['build_hist','train','infer'], required=True)
    parser.add_argument('--nbins', type=int, default=NBINS_DEFAULT)
    parser.add_argument('--out', type=str, default='out.png')
    parser.add_argument('--epochs', type=int, default=5)
    parser.add_argument('--batch', type=int, default=8)
    parser.add_argument('--input', type=str, help='input image for inference')
    parser.add_argument('--period', type=int, default=0)
    parser.add_argument('--hist_npz', type=str, default='period_histograms.npz')
    args = parser.parse_args()

    if args.mode == 'build_hist':
        if load_dataset is None:
            raise RuntimeError('datasets library not available')
        ds = load_dataset('/content/HistoricalColor-ECCV2012-DecadeDatabase.tar')
        # use train split
        build_period_histograms(ds['train'], nbins=args.nbins, out_npz=args.hist_npz)
        return

    if args.mode in ['train','infer']:
        if load_dataset is None:
            raise RuntimeError('datasets library not available')
        ds = load_dataset('/content/HistoricalColor-ECCV2012-DecadeDatabase.tar')
        # choose split
        split = ds['train']
        num_periods = max([int(item.get('label',0)) for item in split]) + 1
        print(f"Loaded dataset with approx {len(split)} images, detected {num_periods} period labels")

    if args.mode == 'train':
        hf_dataset = HistoricalColorizationDataset(split, image_size=IMG_SIZE)
        loader = DataLoader(hf_dataset, batch_size=args.batch, shuffle=True, num_workers=4)
        model = UNetConditional(num_periods=num_periods).to(DEVICE)
        optimizer = torch.optim.Adam(model.parameters(), lr=2e-4)
        # load period hist if exists
        period_hist = {}
        if os.path.exists(args.hist_npz):
            d = np.load(args.hist_npz)
            for k in d.files:
                period_hist[int(k)] = d[k]
        else:
            # fallback: uniform
            for i in range(num_periods):
                period_hist[i] = np.ones((args.nbins*args.nbins,)) / (args.nbins*args.nbins)
        for e in range(args.epochs):
            train_loop(model, loader, optimizer, e, period_hist=period_hist, nbins=args.nbins)
            torch.save(model.state_dict(), f"colorize_period_epoch{e}.pth")
        print("Training complete")

In [28]:
from datasets import load_dataset
import os

try:
    # Download the parquet file
    ds = load_dataset(repo_id="/content/HistoricalColor-ECCV2012-DecadeDatabase.tar", filename="data/train-00000-of-00001.parquet", repo_type="dataset")
    print(f"Historical Photos")

    # Load the dataset from the downloaded parquet file
    dataset = load_dataset("parquet", data_files='/content/HistoricalColor-ECCV2012-DecadeDatabase.tar', split="train")
    print("\nDataset loaded successfully:")
    print(dataset)

except Exception as e:
    print(f"Error loading dataset: {e}")
    print("Please check the dataset page on HistoricalColor")

Error loading dataset: load_dataset() missing 1 required positional argument: 'path'
Please check the dataset page on HistoricalColor


In [29]:
import tarfile

tar_path = "/content/HistoricalColor-ECCV2012-DecadeDatabase.tar"  # your file

try:
    with tarfile.open(tar_path, 'r') as tar:
        tar.extractall("/content/historical_dataset")
    print(" Dataset extracted successfully!")

except Exception as e:
    print(" Error extracting tar file:", e)


/tmp/ipython-input-223184831.py:7: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall("/content/historical_dataset")


 Error extracting tar file: unexpected end of data


In [30]:
from datasets import load_dataset
import glob

# Find parquet files inside extracted folder
parquet_files = glob.glob("/content/HistoricalColor-ECCV2012-DecadeDatabase.tar.parquet", recursive=True)

print("Found parquet files:", parquet_files)

if len(parquet_files) == 0:
    print("No parquet files found!")
else:
    dataset = load_dataset("parquet", data_files=parquet_files)
    print("Dataset loaded successfully!")
    print(dataset)


Found parquet files: []
No parquet files found!


In [34]:
import tarfile

tar_path = "/content/HistoricalColor-ECCV2012-DecadeDatabase.tar"

try:
    with tarfile.open(tar_path, "r") as tar:
        tar.extractall("/content/historical_dataset")
    print("Extraction complete!")

except Exception as e:
    print("Error extracting tar file:", e)


/tmp/ipython-input-946270397.py:7: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall("/content/historical_dataset")


Error extracting tar file: unexpected end of data


In [35]:
import os

root = "/content/historical_dataset"

for path, dirs, files in os.walk(root):
    for f in files:
        print(os.path.join(path, f))


/content/historical_dataset/._HistoricalColor-ECCV2012
/content/historical_dataset/HistoricalColor-ECCV2012/._data
/content/historical_dataset/HistoricalColor-ECCV2012/._COPYRIGHT-DECADE-DATA
/content/historical_dataset/HistoricalColor-ECCV2012/COPYRIGHT-DECADE-DATA
/content/historical_dataset/HistoricalColor-ECCV2012/data/._imgs
/content/historical_dataset/HistoricalColor-ECCV2012/data/imgs/decade_database/1960s/decade_a28cad45e6aafd905732153cedb821cc.jpg
/content/historical_dataset/HistoricalColor-ECCV2012/data/imgs/decade_database/1960s/decade_74ecee8996269e3a79b33db5c987e398.jpg
/content/historical_dataset/HistoricalColor-ECCV2012/data/imgs/decade_database/1960s/decade_89222753d95689678132a8e5efed34e2.jpg
/content/historical_dataset/HistoricalColor-ECCV2012/data/imgs/decade_database/1960s/decade_b5b067f26e3c0d5ee193d65382a5369a.jpg
/content/historical_dataset/HistoricalColor-ECCV2012/data/imgs/decade_database/1960s/decade_d7f48fe25908557a1bc379480f3192ea.jpg
/content/historical_dat

In [37]:
from datasets import load_dataset
import glob

parquet_files = glob.glob("/content/HistoricalColor-ECCV2012-DecadeDatabase.tar.parquet", recursive=True)

if len(parquet_files) == 0:
    print("No parquet files found.")
else:
    dataset = load_dataset("parquet", data_files=parquet_files)
    print("Parquet dataset loaded!")
    print(dataset)


No parquet files found.


In [38]:
from datasets import load_dataset

dataset = load_dataset(
    "imagefolder",
    data_dir="/content/historical_dataset",
    split="train"
)

print("Image dataset loaded!")
print(dataset)


Resolving data files:   0%|          | 0/755 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Image dataset loaded!
Dataset({
    features: ['image', 'label'],
    num_rows: 754
})


In [40]:
from datasets import load_dataset

# The previous cell output showed no CSV files in the extracted directory.
# If you intended to load image data, use the "imagefolder" builder instead.
try:
    dataset = load_dataset(
        "csv",
        data_files="/content/historical_dataset/*.csv" # No CSV files found here
    )

    print("✅ CSV loaded!")
    print(dataset)

except Exception as e:
    print(f"Error loading dataset: {e}")
    print("Please check the file paths and ensure they point to valid CSV files.")

Generating train split: 0 examples [00:00, ? examples/s]

Error loading dataset: An error occurred while generating the dataset
Please check the file paths and ensure they point to valid CSV files.


In [41]:
import os

root = "/content/historical_dataset"

for path, dirs, files in os.walk(root):
    for f in files:
        print(os.path.join(path, f))


/content/historical_dataset/._HistoricalColor-ECCV2012
/content/historical_dataset/HistoricalColor-ECCV2012/._data
/content/historical_dataset/HistoricalColor-ECCV2012/._COPYRIGHT-DECADE-DATA
/content/historical_dataset/HistoricalColor-ECCV2012/COPYRIGHT-DECADE-DATA
/content/historical_dataset/HistoricalColor-ECCV2012/data/._imgs
/content/historical_dataset/HistoricalColor-ECCV2012/data/imgs/decade_database/1960s/decade_a28cad45e6aafd905732153cedb821cc.jpg
/content/historical_dataset/HistoricalColor-ECCV2012/data/imgs/decade_database/1960s/decade_74ecee8996269e3a79b33db5c987e398.jpg
/content/historical_dataset/HistoricalColor-ECCV2012/data/imgs/decade_database/1960s/decade_89222753d95689678132a8e5efed34e2.jpg
/content/historical_dataset/HistoricalColor-ECCV2012/data/imgs/decade_database/1960s/decade_b5b067f26e3c0d5ee193d65382a5369a.jpg
/content/historical_dataset/HistoricalColor-ECCV2012/data/imgs/decade_database/1960s/decade_d7f48fe25908557a1bc379480f3192ea.jpg
/content/historical_dat